<a href="https://colab.research.google.com/github/LolitaSian/Getting-Started-with-Google-BERT/blob/main/Chapter03/3.03.%20Generating%20BERT%20embedding%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face transformers 

Hugging Face is an organization that is on a path to solve and democratize AI through natural language. Their open-source library 'transformers'  is very popular among the NLP community. It is very useful and powerful for several NLP and NLU tasks. It includes thousands of pre-trained models in about 100+ languages. One of the many advantages of the transformer library is that it is compatible with both PyTorch and TensorFlow.

We can install transformers directly using pip as shown in the following: 


In [31]:
!pip install transformers 
# 有原来那句会跑不通，并且指定版本之后会出现奇怪bug，可能是两年前代码太老了。遂改之。

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



As we can see, in this book, we use transformers version 3.5.1. Now that we installed transformers, let's get started. 

# Generating BERT embedding 
In this section, we will learn how to extract embeddings from the pre-trained BERT. Consider the sentence 'I love Paris'. Let's see how to obtain the contextualized word embedding of all the words in the sentence using the pre-trained BERT model with Hugging Face's transformer library. 

First, let's import the necessary modules:

In [32]:
from transformers import BertModel, BertTokenizer
import torch

Next, we download the pre-trained BERT model. We can check all the available pre-trained BERT models here - https://huggingface.co/transformers/pre-trained_models.html.We use the 'bert-base-uncased' model. As the name suggests, it is the BERT-base model with 12 encoders and it is trained with uncased tokens. Since we are using the BERT-base, the representation size will be 768. 

Download and load the pre-trained bert-base-uncasedmodel:

In [33]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Next, we download and load the tokenizer which is used for pretraining the bert-base-uncased model: 

In [34]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Now, let's see how to preprocess the input before feeding it to the BERT. 


## Preprocessing the input 
Define the sentence:

In [35]:
sentence = 'I love Paris'

Tokenize the sentence and obtain the tokens:

In [36]:
tokens = tokenizer.tokenize(sentence)

Let's print the tokens:

In [37]:
print(tokens)

['i', 'love', 'paris']


Now, we will add the [CLS] token at the beginning and [SEP] token at the end of the tokens list: 

In [38]:
tokens = ['[CLS]'] + tokens + ['[SEP]']

Let's look at our updated tokens list:

In [39]:
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]']


As we can observe, we have [CLS] token at the beginning and sep token at the end of our tokens list. We can also observe that length of our tokens is 5.

Say, we need to keep the length of our tokens list to 7, then, in that case, we will add two [PAD] tokens at the end as shown in the following:



In [40]:
tokens = tokens + ['[PAD]'] + ['[PAD]']

Let's print our updated tokens list:

In [41]:
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']




As we can observe, now we have the tokens list consists of [PAD] tokens and the length of our tokens list is 7. 

Next, we create the attention mask. We set the attention mask value to 1 if the token is not a [PAD] token else we will set the attention mask to 0 as shown below:



In [42]:
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]

Let's print the attention_mask:

In [43]:
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]



As we can observe, we have attention mask values 0 at the position where have [PAD] token and 1 at other positions. 

Next, we convert all the tokens to their token_ids as shown below: 


In [44]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)


Let's have a look at the token_ids:

In [45]:
print(token_ids)

[101, 1045, 2293, 3000, 102, 0, 0]



From the above output, we can observe that each token is mapped to a unique token id.

Now, we convert the token_ids and attention_mask to tensors as shown below:

In [46]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)


That's it. Next, we feed the token_ids and attention_mask to the pre-trained BERT model and get the embedding. 

## Getting the embedding 

As shown in the following code, we feed the token_ids, and attention_mask to the model and get the embeddings. Note that the model returns the output as a tuple with two values. The first value indicates the hidden state representation, hidden_rep and it consists of the representation of all the tokens obtained from the final encoder (encoder 12), and the second value, cls_head consists of the representation of the [CLS] token: 

In [47]:
output = model(token_ids, attention_mask = attention_mask)
# 让它自动解包会出问题，所以改了
hidden_rep = output.last_hidden_state
cls_head = output.pooler_output

In the preceding code, hidden_rep contains the embedding(representation) of all the tokens in our input. Let's print the shape of hidden_rep tensor:

In [48]:
print(hidden_rep.shape)

torch.Size([1, 7, 768])


The size [1,7,768] indicates the[batch_size, sequence_length, hidden_size].

Our batch size is 1, the sequence length is the token length, since we have 7 tokens, the sequence length is 7, and the hidden size is the representation (embedding) size and it is 768 for the BERT-base model. 

We can obtain the representation of each token as: 

- hidden_rep[0][0] gives the representation of the first token which is [CLS]
- hidden_rep[0][1] gives the representation of the second token which is 'I' 
- hidden_repo[0][2] gives the representation of the third token which is 'love' 

In this way, we can obtain the contextual representation of all the tokens. This is basically the contextualized word embeddings of all the words in the given sentence. 

Now, let's take a look at the cls_head. It contains the representation of the [CLS] token. Let's print the shape of cls_head :

In [49]:
print(cls_head.shape)

torch.Size([1, 768])


The above code will print:

torch.Size([1,768])

The size [1,768] indicates the[batch_size, hidden_size].

We learned that cls_head holds the aggregate representation of the sentence, so we can use the cls_head as the representation of the given sentence 'I love Paris'. 

We learned how to extract embeddings from the pre-trained BERT. But these are the embeddings obtained only from the topmost encoder layer of BERT which is encoder 12. Can we also extract the embeddings from all the encoder layers of BERT? Yes! We will find out how to do that in the next section. 